<a href="https://colab.research.google.com/github/GustaveRw/NLP-Fellowship/blob/master/Assignment_%7C_Jean_Marie_Gustave_MBONYINSHUTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASSIGNMENT

Installing Transformers and datasets

In [ ]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 451 kB 64.6 MB/s 
     |████████████████████████████████| 7.6 MB 30.9 MB/s 
     |████████████████████████████████| 182 kB 49.5 MB/s 
     |████████████████████████████████| 132 kB 49.9 MB/s 
     |████████████████████████████████| 212 kB 48.1 MB/s 
     |████████████████████████████████| 127 kB 52.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Installing evaluate

In [ ]:
! pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 


Download and Load Dataset

In [ ]:
from datasets import load_dataset

kinyanews = load_dataset("kinnews_kirnews", "kinnews_cleaned")

Generating train split:   0%|          | 0/17014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4254 [00:00<?, ? examples/s]

Dataset kinnews_kirnews downloaded and prepared to /root/.cache/huggingface/datasets/kinnews_kirnews/kinnews_cleaned/0.0.0/f51a8e4cd08b53f8b08510bbb9d07edc38e9cd7032ffaea2216952be87f35050. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Preprocessing

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

A preprocessing function

In [47]:
def preprocess_function(examples): # add other processing
    return tokenizer(examples['title'], truncation=True)

Tokenize

In [48]:
tokenized_kinyanews = kinyanews.map(preprocess_function, batched=True)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to compute to calculate the accuracy:

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Train

Before starting training your model, create a map of the expected ids to their labels with id2label and label2id

In [27]:
id2label = {0: "politics", 1: "sport", 2: "economy", 3: "health", 4: "entertainment", 5: "history", 6: "technology", 7: "culture", 8: "religion", 9: "environment", 10: "education", 11: "relationship"}
label2id = {"politics":0 , "sport":1, "economy":2, "health":3, "entertainment": 4,  "history": 5, "technology": 6, "culture": 7, "religion": 8, "environment":9,  "education": 10, "relationship": 11}

finetuning

In [55]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=12, id2label=id2label, label2id=label2id
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "politics",
    "1": "sport",
    "2": "economy",
    "3": "health",
    "4": "entertainment",
    "5": "history",
    "6": "technology",
    "7": "culture",
    "8": "religion",
    "9": "environment",
    "10": "education",
    "11": "relationship"
  },
  "initializer_range": 0.02,
  "label2id": {
    "culture": 7,
    "economy": 2,
    "education": 10,
    "entertainment": 4,
    "environment": 9,
    "health": 3,
    "history": 5,
    "politics": 0,
    "relationship": 11,
    "religion": 8,
    "sport": 1,
    "technology": 6
  },
  "max_position_em

At this point, only three steps remain:

Define your training hyperparameters in TrainingArguments. The only required 
parameter is output_dir which specifies where to save your model. You'll push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the Trainer will evaluate the accuracy and save the training checkpoint.
Pass the training arguments to Trainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.
Call train() to finetune your model.

In [60]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_kinyanews["train"],
    eval_dataset=tokenized_kinyanews["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: title, content. If title, content are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 17014
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2128
  Number of trainable parameters = 66962700


Epoch,Training Loss,Validation Loss


IndexError: ignored